## Exercise practice and Tests 

In [247]:
import matplotlib

In [248]:
version(matplotlib)

NameError: name 'version' is not defined

In [250]:
matplotlib.version()

AttributeError: module 'matplotlib' has no attribute 'version'

In [251]:
matplotlib.__version__

'3.1.0'

In [255]:
import math
import requests
import numpy as np

pricelist_url = "https://cloudpricingcalculator.appspot.com/static/data/pricelist.json"

In [256]:
r = requests.get(pricelist_url)
pricelist = r.json()
print("Pricing as of {}".format(pricelist['updated']))

Pricing as of 04-March-2020


In [257]:
class HostedCourse(dict):
    def __init__(self, data, pricelist, **kw):
        dict.__init__(self)
        self.update(data)
        self.pricelist = pricelist
        self.update(kw['update'])

    def monthly_cost_all_pd(self):
        pd_total_size = self['num_users'] * self['pd_size_gb']
        pd_rate_gb_month = self.pricelist['gcp_price_list'][self['pd_type']][self['node_region']]
        return pd_rate_gb_month * pd_total_size

    def monthly_cost_node(self):
        node_rate = self.pricelist['gcp_price_list'][self['node_type']][self['node_region']]
        return node_rate * self['sustained_use_factor'] * self['node_monthly_uptime_h']

    def pods_per_node(self):
        mem_per_node = int(self.pricelist['gcp_price_list'][self['node_type']]['memory'])
        return math.floor((mem_per_node-1)/self['mem_per_pod_gb'])
    
    def derived_node_count(self):
        return math.ceil(self['num_active_pods'] / self.pods_per_node())
    
    def monthly_cost_all_nodes(self):
        return self.derived_node_count() * self.monthly_cost_node()
    
    def monthly_cost_total(self):
        return self.monthly_cost_all_nodes() + self.monthly_cost_all_pd()
    
    def monthly_cost_per_student(self):
        return self.monthly_cost_total() / self['num_users']

In [286]:
# defaults 
dsep = {
    ## GKE
    'node_region': 'us',

    # persistent disk
    'pd_type': 'CP-COMPUTEENGINE-STORAGE-PD-SSD',

    # nodes
    # CP-COMPUTEENGINE-VMIMAGE-N1-HIGHMEM-4: 4 cores, 26GB
    # CP-COMPUTEENGINE-VMIMAGE-N1-STANDARD-4: 4 cores, 15GB
    'node_type': 'CP-COMPUTEENGINE-VMIMAGE-N1-HIGHMEM-4',
    
    'num_users': 1,
    'num_active_pods': 1,

    'pd_size_gb': 10,
    'node_monthly_uptime_h': 30*24,
    'mem_per_pod_gb': 2,
    
    # https://cloud.google.com/compute/#pricing
    # "Sustained Use Discounts
    # Earn up to a 30% net discount for instances that run for an entire month.
    # Compute Engine automatically discounts instances running more than 25% of
    # the days in a month1."
    # This is theoretically represented by data['gcp_price_list']['sustained_use_tiers']
    # which suggests discounts of up to 60% which is at odds with the statement above.
    # We'll assume a sustained discount of not the max of 30%, but of 15%.
    # FIXME: rejigger as a function of node_monthly_uptime_h
    'sustained_use_factor': 0.85, 
}

In [291]:
courses = {
    'DSCI-511': HostedCourse(dsep, pricelist, update={
        'num_users': 100,
        'num_active_pods': 40, # 80th percentile; FIXME: acquire via datadogs?
    }),
}

In [292]:
def show_node_info(nt):
    '''Display a node type's hourly rate and it's memory and core counts.'''
    print("rate: {:.3f}, mem_gb: {}, cores: {}".format(
        pricelist['gcp_price_list'][nt]['us'],
        pricelist['gcp_price_list'][nt]['memory'],
        pricelist['gcp_price_list'][nt]['cores']
    ))

In [293]:
show_node_info('CP-COMPUTEENGINE-VMIMAGE-N1-HIGHMEM-4')
show_node_info('CP-COMPUTEENGINE-VMIMAGE-N1-STANDARD-4')



rate: 0.237, mem_gb: 26, cores: 4
rate: 0.190, mem_gb: 15, cores: 4


In [295]:
for k in sorted(courses.keys()):
    course = courses[k]
    print("{:>12}\ttotal: ${:8.2f}\tper user: ${:6.2f}".format(
        k,
        course.monthly_cost_total(),
        course.monthly_cost_per_student()
    ))

    DSCI-511	total: $  749.69	per user: $  7.50
